In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
#Directory variables

PROTEIN_SCORE_DIR = 'fuzzy_python_results'
REDUCED_PROTEIN_SCORE_DIR_OB = 'vprec_ob_results'
REDUCED_PROTEIN_SCORE_DIR_IB = 'vprec_ib_results'
IEEE_DIR = 'IEEE'
IEEE_PROTEIN_SCORE_REF = 'ieee_ref.pkl'
INSTR_LIB = 'py'
THRESHOLD = 0.3

## Restructure protein results to calculate metrics

In [ ]:
new_dict = {}
f=open('data-1.0.6/data/predictions.pkl', 'rb')
predictions = pickle.load(f)

for i in range(1,10):
    new_dict[i] = {}
    for j in range(0,646):
        f = open(PROTEIN_SCORE_DIR + '/%s_%s.pkl'%(i,j), 'rb')
        pkl_dict = pickle.load(f)
        
        for protein in pkl_dict.keys():
            new_dict[i][protein] = pkl_dict[protein]

for i in new_dict:
    
    df = pd.DataFrame({'proteins': new_dict[i].keys(), 'prop_annotations':[list(predictions.loc[predictions['proteins'] == x]['prop_annotations'])[0] for x in new_dict[i]], 'preds':[list(new_dict[i][x].values()) for x in new_dict[i]]})
    df.to_pickle(PROTEIN_SCORE_DIR + '/pkl_%s.pkl'%i)

#### Reduced precision protein results for outbound mode

In [ ]:
new_dict = {}
f=open('data-1.0.6/data/predictions.pkl', 'rb')
predictions = pickle.load(f)

for f1 in [23,7]:
    for f2 in [23,10,7]:
        
        new_dict['%s_%s'%(f1,f2)] = {}
        
        for i in range(0,646):
            
            f = open(REDUCED_PROTEIN_SCORE_DIR_OB + '/%s_%s_%s.pkl'%(f1,f2,i), 'rb')
            temp = pickle.load(f)
            
            pkl_dict = {}
            for prot in temp:
                pkl_dict[prot] = {}
                for go_id in temp[prot]:
                    if temp[prot][go_id] >= 0.3: #REMOVE IF NO THRESHOLD
                        pkl_dict[prot][go_id] = np.float64(temp[prot][go_id])
                        
            for protein in pkl_dict.keys():
                new_dict['%s_%s'%(f1,f2)][protein] = pkl_dict[protein]

for i in new_dict:
        df = pd.DataFrame({'proteins': new_dict[i].keys(), 'prop_annotations':[list(predictions.loc[predictions['proteins'] == x]['prop_annotations'])[0] for x in new_dict[i]], 'preds':[list(new_dict[i][x].values()) for x in new_dict[i]]})
        df.to_pickle(REDUCED_PROTEIN_SCORE_DIR_OB + '/%s.pkl'%i)       



#### Reduced precision protein results for inbound mode

In [ ]:
new_dict = {}
f=open('data-1.0.6/data/predictions.pkl', 'rb')
predictions = pickle.load(f)

for f1 in [23,7,2]:
    for f2 in [23,10,7]:
        
        new_dict['%s_%s'%(f1,f2)] = {}
        
        for i in range(0,646):
            
            f = open(REDUCED_PROTEIN_SCORE_DIR_IB + '/%s_%s_%s.pkl'%(f1,f2,i), 'rb')
            temp = pickle.load(f)
            
            pkl_dict = {}
            for prot in temp:
                pkl_dict[prot] = {}
                for go_id in temp[prot]:
                    if temp[prot][go_id] >= 0.3: #REMOVE IF NO THRESHOLD
                        pkl_dict[prot][go_id] = np.float64(temp[prot][go_id])
                        
            for protein in pkl_dict.keys():
                new_dict['%s_%s'%(f1,f2)][protein] = pkl_dict[protein]

for i in new_dict:
        df = pd.DataFrame({'proteins': new_dict[i].keys(), 'prop_annotations':[list(predictions.loc[predictions['proteins'] == x]['prop_annotations'])[0] for x in new_dict[i]], 'preds':[list(new_dict[i][x].values()) for x in new_dict[i]]})
        df.to_pickle(REDUCED_PROTEIN_SCORE_DIR_IB + '/%s.pkl'%i)       



#### IEEE version

In [ ]:
new_dict = {}
f=open('data-1.0.6/data/predictions.pkl', 'rb')
predictions = pickle.load(f)
f = open(IEEE_DIR + '/' + IEEE_PROTEIN_SCORE_REF, 'rb')
pkl_dict = pickle.load(f)

for protein in pkl_dict.keys():
    new_dict[protein] = pkl_dict[protein]
    
    
df = pd.DataFrame({'proteins': new_dict.keys(), 'prop_annotations':[list(predictions.loc[predictions['proteins'] == x]['prop_annotations'])[0] for x in new_dict], 'preds':[list(new_dict[x].values()) for x in new_dict]})

df.to_pickle(IEEE + '/' + 'pickled_ieee_ref.pkl')

## Gather protein results

In [ ]:
docker_pred = {}
for i in range(1,11):
#     for j in range(0,646):
#         if j >= 573 and j <= 599: continue
    f = open(PROTEIN_SCORE_DIR + '/%s.pkl'%(i), 'rb') #modify to point to correct directory

    pkl_dict = pickle.load(f)

#         pkl_dict = {}
#         for prot in temp:
#             pkl_dict[prot] = {}
#             for go_id in temp[prot]:
#                 if temp[prot][go_id] >= THRESHOLD:
#                     pkl_dict[prot][go_id] = np.float64(temp[prot][go_id])

    for protein in pkl_dict.keys():
        if i == 1:
            docker_pred[protein] = {}
            for go_id in pkl_dict[protein]:
                docker_pred[protein][go_id] = np.array([pkl_dict[protein][go_id]])
        else:

            for go_id in pkl_dict[protein]:
                docker_pred[protein][go_id] = np.append(docker_pred[protein][go_id], pkl_dict[protein][go_id])

original_file = open(IEEE_DIR + '/' + IEEE_PROTEIN_SCORE_REF, 'rb')
pkl_dict = pickle.load(original_file)
original_results = {}

for protein in pkl_dict:
    original_results[protein] = pkl_dict[protein]


## Restructure references for significant digits calculation

#### Protein scores

In [ ]:
multi_iter = {}
for num, protein in enumerate(sorted(list(original_results.keys()))):
    multi_iter[protein] = np.array(list(original_results[protein].values()))

np.save('References/ref_deepgoplus_proteins_' + INSTR_LIB, np.array(list(multi_iter.values()))) 

#### Metrics

In [ ]:
for i in ['bp', 'cc', 'mf']:
    metric_file = open(IEEE_DIR + '/%s.txt'%i)
    ref = open('References/ref_%s.txt'%i, 'w') # open('significant_digits2/References/ref_%s_results.txt'%i, 'w')

    for i, row in enumerate(metric_file):
        if i == 0: continue
        ref.write(row.split(': ')[-1])
    
    ref.close()

## Restructure results for significant digits calculation

#### Protein scores

In [ ]:
multi_iter = {}
for num, protein in enumerate(sorted(list(docker_pred.keys()))):

    ref = np.array(list(docker_pred[protein].values()))
    temp = []
    for i in range(10):
        temp.append(np.array(ref[:,i]))
    multi_iter[protein] = np.array(temp)
        
#modify names according to what results are being restructured
f = open('References/deepgoplus_proteins_%s.pkl'%INSTR_LIB, 'wb')
pickle.dump(multi_iter,f)

#### Metrics

In [ ]:
ont_array = {}
for ont in ['bp', 'cc', 'mf']:
    metric_array = []
    for num in range(1,11):
        metric_file = open(PROTEIN_SCORE_DIR + '/%s_%s.txt'%(ont, num))
        
        for i, row in enumerate(metric_file):
#             if 'Length' in row or 'Precision' in row or 'Recall' in row or 'Positive' in row or 'Negative' in row or 'Smin threshold' in row: continue
            if 'Length' in row: continue
        
            metric_array.append(np.float64(row.split(': ')[-1]))

    ont_array[ont] = np.array(metric_array).reshape(10,4)
    
    #modify names according to what metrics are being restructured
    np.save('References/%s_%s'%(ont, INSTR_LIB), ont_array[ont])